In [1]:
import numpy as np
import os
import nrrd
import h5py
import pandas

/home/davech2y/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
database_root = "/mnt/raid/davech2y/ShapeNetCore_vol/"
database_name = "nrrd_256_filter_div_32_solid.hdf5"
database = h5py.File(os.path.join(database_root, database_name), "w")

In [3]:
shapenet_root = "/mnt/raid/davech2y/ShapeNetCore_vol/nrrd_256_filter_div_32_solid/"
shapenet_csv = pandas.read_csv("captions.tablechair.csv")
shapenet_shapepath = [
    os.path.join(shapenet_root, model_name, model_name + '.nrrd') for model_name in shapenet_csv.modelId.values.tolist()
]
shapenet_size = len(shapenet_shapepath)
shapenet_shape = [3, 32, 32, 32]
shapenet_shape_flat = np.prod(shapenet_shape)

In [4]:
shapenet_dataset = database.create_dataset("shapes", (shapenet_size, shapenet_shape_flat), dtype="float")
# shapenet_database = database["shapes"]

In [7]:
for i, shapepath in enumerate(shapenet_shapepath):
    if i % 100 == 0: 
        print("preprocessed and stored: ", i)
    shape, _ = nrrd.read(shapepath)
    shape = np.reshape(shape[:3], (-1))
    shape = (shape - np.min(shape)) / (np.max(shape) - np.min(shape))
    shapenet_dataset[i] = shape